In [1]:
import numpy as np

In [6]:
from datasets import load_dataset
snli = load_dataset("snli")
#Removing sentence pairs with no label (-1)
snli = snli.filter(lambda example: example['label'] != -1) 

# 0 : entails
# 1 : neutral
# 2 : contradicts

In [19]:
labels = snli['test']['label']
labels = np.array(labels)

In [25]:
BERTs = [
    np.load('bert-large-uncased_20240327_173829.npy'),
    np.load('bert-large-uncased_20240327_103627.npy'),  
    np.load('bert-large-uncased_20240327_123313.npy')
]

RoBERTas = [
    np.load('roberta-large_20240327_174024.npy'),
    np.load('roberta-large_20240327_103801.npy'),
    np.load('roberta-large_20240327_103720.npy')
]
# print(bert.shape) = (9824, 3)

In [26]:
# softmax : 
sm_bert = [
    np.exp(bert) / np.sum(np.exp(bert), axis=1)[:, np.newaxis]
    for bert in BERTs
]
sm_roberta = [
    np.exp(roberta) / np.sum(np.exp(roberta), axis=1)[:, np.newaxis]
    for roberta in RoBERTas
]

model = sm_bert[0]
for i in range(1, 3):
    model += sm_bert[i]

for i in range(3):
    model += sm_roberta[i]

model /= 6


In [29]:
model_labels = np.argmax(model, axis=1)
acc = (model_labels == labels).mean()
acc_roberta = [(np.argmax(sm, axis=1) == labels).mean() for sm in sm_roberta]
acc_bert = [(np.argmax(sm, axis=1) == labels).mean() for sm in sm_bert]

In [30]:
print(f"RoBERTa accuracy: {acc_roberta}")
print(f"BERT accuracy: {acc_bert}")
print(f"Ensemble accuracy: {acc}")

RoBERTa accuracy: [0.9228420195439739, 0.9185667752442996, 0.9177524429967426]
BERT accuracy: [0.9267100977198697, 0.896478013029316, 0.9091001628664495]
Ensemble accuracy: 0.9267100977198697
